In [1]:
import pandas as pd

In [2]:
inFPath = 'budget.xlsx'
df = pd.read_excel(inFPath, sheet_name='forPandas', nrows=5)

In [3]:
cols = [c for c in df.columns if 'unnamed' not in c.lower()]
df = df[cols]
df

,Year1 FTE,Year1 Months,Year1 Unloaded,Year1 Overheads,Year1 Totals,Year2 FTE,Year2 Months,Year2 Unloaded,Year2 Overheads,Year2 Totals,...,Year4 FTE,Year4 Months,Year4 Unloaded,Year4 Overheads,Year4 Totals,Year5 FTE,Year5 Months,Year5 Unloaded,Year5 Overheads,Year5 Totals
0,0.683333,8.2,121622,36486.000,158108.000,0.516667,6.2,96072,28821.000,124893.000,...,0.5,6,109687,32906.000,142593.000,0.50,6,113932,34179,148111
1,1.083333,13.0,75993,22798.000,98791.000,1.666667,20.0,122225,36667.000,158892.000,...,2.0,24,159220,47766.000,206986.000,1.25,15,103701,31110,134811
2,0.000000,0.0,21000,4462.773,25462.773,0.000000,0.0,0,0.000,0.000,...,0.0,0,0,0.000,0.000,0.00,0,0,0,0
3,0.000000,0.0,6000,1275.078,7275.078,0.000000,0.0,18000,3825.234,21825.234,...,0.0,0,21000,4462.773,25462.773,0.00,0,16500,0,16500
4,1.766667,21.2,224615,65021.851,289636.851,2.183333,26.2,236297,69313.234,305610.234,...,2.5,30,289907,85134.773,375041.773,1.75,21,234133,65289,299422


In [19]:
# Personnel budget
personnel = df.iloc[0:2]
cols = [c for c in personnel.columns if 'FTE' not in c]
personnel = personnel[cols]
personnelYear = []
budgetDF = 0
for i in range(1,6):
    cols = [c for c in personnel.columns if f'Year{i}' in c]
    tempDF = personnel[cols]
    newColNames = {}
    for col in cols:
        newColNames[col] = col.replace(f'Year{i} ','PI ')

    tempDF = tempDF.rename(columns=newColNames)
    postDocDF = tempDF.iloc[1]
    newDF = tempDF.drop(1)
    for col in tempDF.keys():
        postdocColName = col.replace('PI', 'Postdoc')
        newDF[postdocColName] = postDocDF[col]
        newDF = newDF.rename(index={0:f'Period {i}'})
        
    personnelYear.append(newDF)
personnelDF = pd.concat(personnelYear)
personnelDF = personnelDF.round(2)

format_mapping={}
for col in personnelDF:
    if 'month' in col.lower():
        format_mapping[col] = '{:,.1f}'
    else:
        format_mapping[col] = '${:,.0f}'
        
personnelDF.loc['Total']= personnelDF.sum(axis=0)
personnelDF = personnelDF.round(2)

for key, value in format_mapping.items():
    personnelDF[key] = personnelDF[key].apply(value.format)
cols = [col for col in personnelDF.keys() if 'Overheads' not in col and 'Totals' not in col]
#personnelDF =personnelDF[cols]
personnelDF

,PI Months,PI Unloaded,PI Overheads,PI Totals,Postdoc Months,Postdoc Unloaded,Postdoc Overheads,Postdoc Totals
Period 1,8.2,"$121,622","$36,486","$158,108",13.0,"$75,993","$22,798","$98,791"
Period 2,6.2,"$96,072","$28,821","$124,893",20.0,"$122,225","$36,667","$158,892"
Period 3,6.0,"$105,602","$31,681","$137,283",30.0,"$191,015","$57,305","$248,320"
Period 4,6.0,"$109,687","$32,906","$142,593",24.0,"$159,220","$47,766","$206,986"
Period 5,6.0,"$113,932","$34,179","$148,111",15.0,"$103,701","$31,110","$134,811"
Total,32.4,"$546,915","$164,073","$710,988",102.0,"$652,154","$195,646","$847,800"


In [24]:
personnelLatex = personnelDF.to_latex()
personnelLatex = personnelLatex.replace('PI ', '').replace('Postdoc ', '')
personnelLatex = personnelLatex.replace('Overheads', 'Fringe').replace('Unloaded', 'Salary')
personnelLatex = personnelLatex.replace('\nTotal ', '\n\\hline\nTotal ')

# This can be done better...
personnelLatex = personnelLatex.replace('toprule\n', 'toprule\n & \\multicolumn{4}{c|}{Scientific} & \\multicolumn{4}{c}{Postdoctoral} \\\\\n')
personnelLatex = personnelLatex.replace('\\toprule', '\\hline').replace('\\midrule', '\\hline').replace('\\bottomrule', '\\hline')
# Again, this can be done more elegantly by counting the number of columns fol each type of pelsonnel. Maybe tomorrow...
personnelLatex = personnelLatex.replace('lllllllll', 'lrrrr|rrrr')
print(personnelLatex)
with open('personnelBudget.tex', 'w') as personnelF:  
    personnelF.write(personnelLatex)

\begin{tabular}{lrrrr|rrrr}
\hline
 & \multicolumn{4}{c|}{Scientific} & \multicolumn{4}{c}{Postdoctoral} \\
{} & Months & Salary & Fringe & Totals & Months & Salary & Fringe & Totals \\
\hline
Period 1 &       8.2 &    \$121,622 &      \$36,486 &  \$158,108 &           13.0 &          \$75,993 &           \$22,798 &        \$98,791 \\
Period 2 &       6.2 &     \$96,072 &      \$28,821 &  \$124,893 &           20.0 &         \$122,225 &           \$36,667 &       \$158,892 \\
Period 3 &       6.0 &    \$105,602 &      \$31,681 &  \$137,283 &           30.0 &         \$191,015 &           \$57,305 &       \$248,320 \\
Period 4 &       6.0 &    \$109,687 &      \$32,906 &  \$142,593 &           24.0 &         \$159,220 &           \$47,766 &       \$206,986 \\
Period 5 &       6.0 &    \$113,932 &      \$34,179 &  \$148,111 &           15.0 &         \$103,701 &           \$31,110 &       \$134,811 \\
\hline
Total    &      32.4 &    \$546,915 &     \$164,073 &  \$710,988 &          102.

In [22]:
# Now let's get the travel costs.
cols = [col for col in df.keys() if 'Unloaded' in col]
travelCosts = df[cols].iloc[3]
newColNames = {}
for i, col in enumerate(travelCosts.keys()):
    newColNames[col] = col.replace(f'Year{i+1} Unloaded',f'Budget period {i+1}')
travelCosts = travelCosts.rename(newColNames)
travelCostsDF = travelCosts.to_frame()
travelCostsDF = travelCostsDF.rename(columns={3:'Travel Funds'})
travelCostsDF = travelCostsDF.round(2)
travelCostsDF['Travel Funds'] = travelCostsDF['Travel Funds'].map('${:,.0f}'.format)
# Some magic numbers
travelCostsDF['Domestic trips'] = [0, 2, 3, 0, 3]
travelCostsDF['International trips'] = [2, 5, 6, 7, 4]
travelCostsDF = travelCostsDF[['Domestic trips', 'International trips', 'Travel Funds']]
travelLatex = travelCostsDF.to_latex()

travelLatex = travelLatex.replace('\\toprule', '\\hline').replace('\\midrule', '\\hline').replace('\\bottomrule', '\\hline')
travelLatex = travelLatex.replace('lrrl', 'lrrr')
print(travelLatex)
with open('travelBudget.tex', 'w') as travelF:  
    travelF.write(travelLatex)


\begin{tabular}{lrrr}
\hline
{} &  Domestic trips &  International trips & Travel Funds \\
\hline
Budget period 1 &               0 &                    2 &       \$6,000 \\
Budget period 2 &               2 &                    5 &      \$18,000 \\
Budget period 3 &               3 &                    6 &      \$22,500 \\
Budget period 4 &               0 &                    7 &      \$21,000 \\
Budget period 5 &               3 &                    4 &      \$16,500 \\
\hline
\end{tabular}



In [11]:
# Now for total costs
cols = [col for col in df.keys() if 'Unloaded' in col]
unloaded = df[cols]
totals = unloaded.iloc[4]
total = round(totals.sum(),2)
unloaded['totals'] = unloaded.sum(axis=1)
unloaded = unloaded.round(2)
salarySum = unloaded['totals'].iloc[0]+unloaded['totals'].iloc[1]
outStr = f'The total direct costs are \${total:,.2f} of which \${salarySum:,.2f} is in salaries, \${unloaded["totals"].iloc[2]:,.2f} is for equipment, and \${unloaded["totals"].iloc[3]:,.2f} is for travel.'
print(outStr)
with open('directs.tex', 'w') as directsF:  
    directsF.write(outStr)


The total direct costs are \$1,304,069.00 of which \$1,199,069.00 is in salaries, \$21,000.00 is for equipment, and \$84,000.00 is for travel.


<ipython-input-11-392d3a4f80d6>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unloaded['totals'] = unloaded.sum(axis=1)


In [12]:
cols = [col for col in df.keys() if 'Overheads' in col]
indirects = df[cols]
indirects.loc['Personnel'] = indirects.iloc[0]+indirects.iloc[1]
indirects.loc['Travel and M&S'] = indirects.iloc[2]+indirects.iloc[3]
indirects = indirects.iloc[5:]
newColNames = {}
for i, col in enumerate(indirects.keys()):
    newColNames[col] = col.replace(f'Year{i+1} Overheads',f'Budget period {i+1}')
indirects = indirects.rename(columns=newColNames)
indirects = indirects.transpose()
indirects.loc['Total'] = indirects.sum(axis=0)
indirectsFormatted = indirects.round(2)
for col in indirectsFormatted.keys():
    indirectsFormatted[col] = indirectsFormatted[col].map('${:,.2f}'.format)
indirectsLatex = indirectsFormatted.to_latex()

indirectsLatex = indirectsLatex.replace('\\toprule', '\\hline').replace('\\midrule', '\\hline').replace('\\bottomrule', '\\hline')
indirectsLatex = indirectsLatex.replace('lll', 'lrr')
indirectsLatex = indirectsLatex.replace('Total', '\\hline\nTotal')
print(indirectsLatex)
with open('indirects.tex', 'w') as indirectsF:  
    indirectsF.write(indirectsLatex)


\begin{tabular}{lrr}
\hline
{} &    Personnel & Travel and M\&S \\
\hline
Budget period 1 &   \$59,284.00 &      \$5,737.85 \\
Budget period 2 &   \$65,488.00 &      \$3,825.23 \\
Budget period 3 &   \$88,986.00 &      \$4,781.54 \\
Budget period 4 &   \$80,672.00 &      \$4,462.77 \\
Budget period 5 &   \$65,289.00 &          \$0.00 \\
\hline
Total           &  \$359,719.00 &     \$18,807.40 \\
\hline
\end{tabular}



/usr/local/lib/python3.9/site-packages/pandas/core/indexing.py:692: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value, self.name)


In [13]:
newColNames = {}
for i, col in enumerate(unloaded.keys()):
    newColNames[col] = col.replace(f'Year{i+1} Unloaded',f'Budget period {i+1}')
directs = unloaded.rename(columns=newColNames)
directs = directs.transpose()
totalDirects = directs[4]
totalIndirects = indirects.sum(axis=1)
totalCosts = totalDirects.to_frame()
totalCosts = totalCosts.rename(columns={4:'Direct costs'})
totalCosts['Indirect costs'] = totalIndirects
totalCosts = totalCosts.rename({'totals':'Total'})
totalCosts.loc['Total']['Indirect costs'] = totalCosts['Indirect costs'].sum(axis=0)
totalCosts['Total cost'] = totalCosts.sum(axis=1)

totalCostsFormatted = totalCosts.round(2)
for col in totalCostsFormatted.keys():
    totalCostsFormatted[col] = totalCostsFormatted[col].map('${:,.2f}'.format)
totalCostsLatex = totalCostsFormatted.to_latex()

totalCostsLatex = totalCostsLatex.replace('\\toprule', '\\hline').replace('\\midrule', '\\hline').replace('\\bottomrule', '\\hline')
totalCostsLatex = totalCostsLatex.replace('lrrl', 'lrrr')
totalCostsLatex = totalCostsLatex.replace('\nTotal', '\\hline\nTotal')
print(totalCostsLatex)
with open('totalCosts.tex', 'w') as totalCostsF:  
    totalCostsF.write(totalCostsLatex)
outStr = f'The total cost of this proposal is {totalCostsFormatted["Total cost"].loc["Total"]}. '.replace("$","\\$")
with open('totalCostsText.tex', 'w') as totalCostsF:  
    totalCostsF.write(outStr)

\begin{tabular}{llll}
\hline
{} &   Direct costs & Indirect costs &     Total cost \\
\hline
Budget period 1 &    \$224,615.00 &     \$65,021.85 &    \$289,636.85 \\
Budget period 2 &    \$236,297.00 &     \$69,313.23 &    \$305,610.23 \\
Budget period 3 &    \$319,117.00 &     \$93,767.54 &    \$412,884.54 \\
Budget period 4 &    \$289,907.00 &     \$85,134.77 &    \$375,041.77 \\
Budget period 5 &    \$234,133.00 &     \$65,289.00 &    \$299,422.00 \\\hline
Total           &  \$1,304,069.00 &           \$nan &  \$1,304,069.00 \\
\hline
\end{tabular}



<ipython-input-13-8791b8f65c2b>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  totalCosts.loc['Total']['Indirect costs'] = totalCosts['Indirect costs'].sum(axis=0)
